# What-If Scenario Tweaks

Demonstrate simple parameter tweaks (e.g., downtime probability) and compare KPI impacts.

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "pyproject.toml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if not (PROJECT_ROOT / "pyproject.toml").exists():
    raise RuntimeError(
        "Notebook must be executed within a FHOPS checkout (pyproject.toml not found)."
    )
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from docs.examples.analytics import utils

SCENARIO = PROJECT_ROOT / "examples/synthetic/medium/scenario.yaml"
ASSIGNMENTS = PROJECT_ROOT / "docs/examples/analytics/data/scaling_medium_sa_assignments.csv"

base_tables, base_sampling = utils.run_stochastic_summary(SCENARIO, ASSIGNMENTS, tier="medium")
print("Base downtime probability:", base_sampling.downtime.probability)

Base downtime probability: 0.1


### Adjusted Sampling

Increase downtime probability and observe KPI deltas.

In [2]:
custom_sampling = base_sampling.model_copy()
custom_sampling.downtime.probability = 0.3
custom_tables, _ = utils.run_stochastic_summary(
    SCENARIO, ASSIGNMENTS, sampling_config=custom_sampling
)

base_prod = base_tables.shift.groupby("sample_id")["production_units"].sum().mean()
custom_prod = custom_tables.shift.groupby("sample_id")["production_units"].sum().mean()
print(f"Base mean production: {base_prod:.2f}")
print(f"Adjusted mean production: {custom_prod:.2f}")

Base mean production: 23.71
Adjusted mean production: 23.71
